In [1]:
# 安装依赖
!python -m pip install parquet-tools pandas pyarrow > pip_intall.log

本文尝试的是如何定义 Parquet 的 Schema, 然后据此填充数据并生成 Parquet 文件。 

将演示两个例子，一个是没有层级的两个字段，另一个是含于嵌套级别的字段，将要使用到的 Python 模块有 pandas 和 pyarrow

# 简单字段定义
## 定义 Schema 并生成 Parquet 文件

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# 定义 Schema
schema = pa.schema([
    ('id', pa.int32()),
    ('email', pa.string())
])

# 准备数据
ids = pa.array([1, 2], type=pa.int32())
emails = pa.array(['first@example.com', 'second@example.com'], type=pa.string())

# 生成 Parquet 数据
batch = pa.RecordBatch.from_arrays(
    [ids, emails],
    schema = schema
)
table = pa.Table.from_batches([batch])

# 写 Parquet 文件 plain.parquet
pq.write_table(table, 'plain.parquet')

## 写 Parquet 文件 plain.parquet
```
pq.write_table(table, ‘plain.parquet’ )
```

## 验证 Parquet 数据文件
我们可以用工具 parquet-tools 来查看 plain.parquet 文件的数据和 Schema

In [3]:
! parquet-tools inspect plain.parquet


############ file meta data ############
created_by: parquet-cpp-arrow version 8.0.0
num_columns: 2
num_rows: 2
num_row_groups: 1
format_version: 1.0
serialized_size: 533


############ Columns ############
id
email

############ Column(id) ############
name: id
path: id
max_definition_level: 1
max_repetition_level: 0
physical_type: INT32
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -5%)

############ Column(email) ############
name: email
path: email
max_definition_level: 1
max_repetition_level: 0
physical_type: BYTE_ARRAY
logical_type: String
converted_type (legacy): UTF8
compression: SNAPPY (space_saved: -3%)



In [4]:
! parquet-tools show plain.parquet

+------+--------------------+
|   id | email              |
|------+--------------------|
|    1 | first@example.com  |
|    2 | second@example.com |
+------+--------------------+


In [5]:
! parquet-tools csv plain.parquet

id,email

1,first@example.com

2,second@example.com




没问题，与我们期望的一致。也可以用 pyarrow 代码来获取其中的 Schema 和数据

In [6]:
schema = pq.read_schema('plain.parquet')
schema

id: int32
email: string

In [7]:
df = pd.read_parquet('plain.parquet')
df.to_json()

'{"id":{"0":1,"1":2},"email":{"0":"first@example.com","1":"second@example.com"}}'

In [8]:
df

,id,email
0,1,first@example.com
1,2,second@example.com


# 含嵌套字段定义
下面的 Schema 定义加入一个嵌套对象，在 address 下分 email_address 和 post_address，Schema 定义及生成 Parquet 文件的代码如下

In [9]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# 内部字段
address_fields = [
    ('email_address', pa.string()),
    ('post_address', pa.string()),
]

# 定义 Parquet Schema，address 嵌套了 address_fields
schema = pa.schema([
    ('id', pa.int32()),
    ('email', pa.struct(address_fields))
])

# 准备数据
ids = pa.array([1, 2], type = pa.int32())
addresses = pa.array(
    [('first@example.com', 'city1'), ('second@example.com', 'city2')],
    pa.struct(address_fields)
)

# 生成 Parquet 数据
batch = pa.RecordBatch.from_arrays(
    [ids, addresses],
    schema = schema
)
table = pa.Table.from_batches([batch])

# 写 Parquet 数据到文件
pq.write_table(table, 'nested.parquet')

# 验证 Parquet 数据文件
同样用 parquet-tools 来查看下 nested.parquet 文件


In [10]:
! parquet-tools inspect nested.parquet


############ file meta data ############
created_by: parquet-cpp-arrow version 8.0.0
num_columns: 3
num_rows: 2
num_row_groups: 1
format_version: 1.0
serialized_size: 819


############ Columns ############
id
email_address
post_address

############ Column(id) ############
name: id
path: id
max_definition_level: 1
max_repetition_level: 0
physical_type: INT32
logical_type: None
converted_type (legacy): NONE
compression: SNAPPY (space_saved: -5%)

############ Column(email_address) ############
name: email_address
path: email.email_address
max_definition_level: 2
max_repetition_level: 0
physical_type: BYTE_ARRAY
logical_type: String
converted_type (legacy): UTF8
compression: SNAPPY (space_saved: -3%)

############ Column(post_address) ############
name: post_address
path: email.post_address
max_definition_level: 2
max_repetition_level: 0
physical_type: BYTE_ARRAY
logical_type: String
converted_type (legacy): UTF8
compression: SNAPPY (space_saved: -5%)



In [11]:
! parquet-tools show nested.parquet

+------+------------------------------------------------------------------+
|   id | email                                                            |
|------+------------------------------------------------------------------|
|    1 | {'email_address': 'first@example.com', 'post_address': 'city1'}  |
|    2 | {'email_address': 'second@example.com', 'post_address': 'city2'} |
+------+------------------------------------------------------------------+


In [12]:
! parquet-tools csv nested.parquet

id,email

1,"{'email_address': 'first@example.com', 'post_address': 'city1'}"

2,"{'email_address': 'second@example.com', 'post_address': 'city2'}"




用 parquet-tools 看到的 Schama 并没有 struct 的字样，但体现了它 address 与下级属性的嵌套关系。

用 pyarrow 代码来读取 nested.parquet 文件的 Schema 和数据是什么样子

In [13]:
schema = pq.read_schema("nested.parquet")
schema

id: int32
email: struct<email_address: string, post_address: string>
  child 0, email_address: string
  child 1, post_address: string

In [14]:
df = pd.read_parquet('nested.parquet')
df.to_json()

'{"id":{"0":1,"1":2},"email":{"0":{"email_address":"first@example.com","post_address":"city1"},"1":{"email_address":"second@example.com","post_address":"city2"}}}'

In [15]:
df

,id,email
0,1,"{'email_address': 'first@example.com', 'post_a..."
1,2,"{'email_address': 'second@example.com', 'post_..."


数据当然是一样的，有略微不同的是显示的 Schema 中, address 标识为 struct<email_address: string, post_address: string> , 明确的表明它是一个 struct 类型，而不是只展示嵌套层次。

最后留下一个问题，前面我们定义 Parquet Schema 都是在 Python 代码中完成了，Parquet 是否也能像 Avro 一样用外部文件来定义 Schema, 然后编译给 Python 用？